In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
# import numpy as np

# pd_jupiter = pd.read_excel('./jupyter.xlsx', engine='openpyxl')
# pd_X = pd_jupiter[['x1', 'x2', 'x3']]
# pd_Y = pd_jupiter.y
# train_X, test_X, train_Y, test_y = train_test_split(pd_X, pd_Y, test_size=0.1, random_state=42)

# treeCLF = RandomForestClassifier(random_state=0, min_samples_split=2, min_samples_leaf=1)
# parameters = {
#     'n_estimators': range(1, 5),
#     'max_depth': range(3, 5),
# }
# cv = [(slice(None), slice(None))]

# searchTree = GridSearchCV(treeCLF, parameters, cv=cv)
# searchTree.fit(train_X, train_Y)
# bestEstimatorTree = searchTree.best_estimator_
# pred_y = bestEstimatorTree.predict(test_X)
# test_y = test_y.reset_index(drop=True)
# resultDatasetPredictions = pd.DataFrame({
#     'test_y': test_y,
#     'pred_y': pred_y,
#     'diff': pred_y - test_y,
# })
# # resultDatasetPredictions
# joblib.dump(bestEstimatorTree, './jupyter/jupyter.pkl')
#pd_jupiter = pd.read_csv('./jupyter.csv')
#pd_jupiter.iloc[3]

In [2]:



# and later you can load it
# clf = joblib.load('filename.pkl')

In [ ]:
def sayHello():
    return pd.DataFrame(data={'x1': [1, 2], 'x2': [3, 5], 'x3': [8, 16]})

# Рабочее пространство



## Загрузка exel

In [3]:
#Создание ДатаФраймов 
dep_xlsx_vk = pd.read_excel("jupyter/dep_vk.xlsx")
dep_xlsx_tg = pd.read_excel("jupyter/dep_tg.xlsx")
dep_csv = pd.read_csv("jupyter/dep_user.csv")

In [4]:
#Исследуемые параметры
parameters_vk = ["Likes","Reposts","Comments","Views","ER","Text","VR","Date","Media 1"]
parameters_tg = ["Views","Text","VR","Date","Media 1"]
parameters_csv_gender = ['gender', 'contract_sum', "purchase_sum", "purchase_count"]
parameters_csv_city = ['city', 'contract_sum', "purchase_sum", "purchase_count"]
parameters_csv_birth_date = ['birth_date', 'contract_sum', "purchase_sum", "purchase_count"]
#Название параметров
rename_parameters_vk = ['likes','reposts','comments','views','coef_vov','title','coef_ohv','datetime_post','media']
rename_parameters_tg = ['views','title','coef_ohv','datetime_post','media']
#Исследуемые параметры
investigated_parameters_vk = ['datetime_post','likes','comments','views','reposts','coef_vov','coef_ohv']
investigated_parameters_tg = ['datetime_post','views','coef_ohv']

In [5]:
#Очистка Датафрейма
dep_xlsx_vk = dep_xlsx_vk[parameters_vk]
dep_xlsx_tg = dep_xlsx_tg[parameters_tg]

In [6]:
#Переименование параметров
#ВК
for i in range(len(parameters_vk)):
    dep_xlsx_vk.rename(columns={parameters_vk[i]: rename_parameters_vk[i]}, inplace=True)
#ТГ
for i in range(len(parameters_tg)):
    dep_xlsx_tg.rename(columns={parameters_tg[i]: rename_parameters_tg[i]}, inplace=True)

In [7]:
#Группировка параметров по дате
#ВК
dep_xlsx_vk_date = dep_xlsx_vk.copy()
dep_xlsx_vk_date['datetime_post'] = pd.to_datetime(dep_xlsx_vk['datetime_post']).dt.date
dep_xlsx_vk_date = dep_xlsx_vk_date.groupby('datetime_post').sum()
#ТГ
dep_xlsx_tg_date = dep_xlsx_tg.copy()
dep_xlsx_tg_date['datetime_post'] = pd.to_datetime(dep_xlsx_tg['datetime_post']).dt.date
dep_xlsx_tg_date = dep_xlsx_tg_date.groupby('datetime_post').sum()

In [8]:
#Cоздание ДатаФрейма для группировки данных по интервалам дня(утро, день, вечер, ночь)
#ВК
dep_xlsx_vk_date_interval = pd.DataFrame({'interval': ['night', 'morning', 'snack', 'lunch', 'dinner']})
investigated_parameters_vk.remove('datetime_post')
#ТГ
dep_xlsx_tg_date_interval = pd.DataFrame({'interval': ['night', 'morning', 'snack', 'lunch', 'dinner']})
investigated_parameters_tg.remove('datetime_post')

#Заполнение ДатаФрейма нулями( - 1, так как нумерация начинается с нуля)
#ВК
for i in range(len(investigated_parameters_vk)):
    dep_xlsx_vk_date_interval[investigated_parameters_vk[i]] = [0] * 5
#ТГ
for i in range(len(investigated_parameters_tg)):
    dep_xlsx_tg_date_interval[investigated_parameters_tg[i]] = [0] * 5


In [9]:
#Группировка параметров по времени суток
#ВК
for i in range(dep_xlsx_vk.shape[0]):
    #Получение чсисла равное часам в данной записи
    hour = dep_xlsx_vk['datetime_post'][i].hour
    if((hour>= 0 and hour < 6) or (hour>= 22 and hour < 24)) :
        for j in range(len(investigated_parameters_vk)):
            dep_xlsx_vk_date_interval[investigated_parameters_vk[j]][0] += dep_xlsx_vk[investigated_parameters_vk[j]][i]                                                                                               
    if(hour>= 6 and hour < 10):
        for j in range(len(investigated_parameters_vk)):
            dep_xlsx_vk_date_interval[investigated_parameters_vk[j]][1] += dep_xlsx_vk[investigated_parameters_vk[j]][i]
    if(hour>= 10 and hour < 14):
        for j in range(len(investigated_parameters_vk)):
            dep_xlsx_vk_date_interval[investigated_parameters_vk[j]][2] += dep_xlsx_vk[investigated_parameters_vk[j]][i]
    if(hour>= 14 and hour < 18):
        for j in range(len(investigated_parameters_vk)):
            dep_xlsx_vk_date_interval[investigated_parameters_vk[j]][3] += dep_xlsx_vk[investigated_parameters_vk[j]][i]
    if(hour>= 18 and hour < 22):
        for j in range(len(investigated_parameters_vk)):
            dep_xlsx_vk_date_interval[investigated_parameters_vk[j]][4] += dep_xlsx_vk[investigated_parameters_vk[j]][i]
#ТГ
for i in range(dep_xlsx_tg.shape[0]):
    #Получение чсисла равное часам в данной записи
    hour = dep_xlsx_tg['datetime_post'][i].hour
    if((hour>= 0 and hour < 6) or (hour>= 22 and hour < 24)) :
        for j in range(len(investigated_parameters_tg)):
            dep_xlsx_tg_date_interval[investigated_parameters_tg[j]][0] += dep_xlsx_tg[investigated_parameters_tg[j]][i]                                                                                               
    if(hour>= 6 and hour < 10):
        for j in range(len(investigated_parameters_tg)):
            dep_xlsx_tg_date_interval[investigated_parameters_tg[j]][1] += dep_xlsx_tg[investigated_parameters_tg[j]][i]
    if(hour>= 10 and hour < 14):
        for j in range(len(investigated_parameters_tg)):
            dep_xlsx_tg_date_interval[investigated_parameters_tg[j]][2] += dep_xlsx_tg[investigated_parameters_tg[j]][i]
    if(hour>= 14 and hour < 18):
        for j in range(len(investigated_parameters_tg)):
            dep_xlsx_tg_date_interval[investigated_parameters_tg[j]][3] += dep_xlsx_tg[investigated_parameters_tg[j]][i]
    if(hour>= 18 and hour < 22):
        for j in range(len(investigated_parameters_tg)):
            dep_xlsx_tg_date_interval[investigated_parameters_tg[j]][4] += dep_xlsx_tg[investigated_parameters_tg[j]][i]

In [10]:
#Группировка по гендеру
dep_csv_gender = dep_csv[parameters_csv_gender]
dep_csv_gender = dep_csv_gender.groupby('gender').sum()

In [11]:
#Группировка по городу
dep_csv_city = dep_csv[parameters_csv_city]
dep_csv_city = dep_csv_city.groupby('city').sum()

In [12]:
#Группировка по дате рождения
dep_csv_birth_date = dep_csv[parameters_csv_birth_date]
dep_csv_birth_date = dep_csv_birth_date.groupby('birth_date').sum()